In [68]:
# 19236697 Sean Ryan and 19255551 Eoin Costello
# Code executes in full without error
# Source we used for the base of writing the code, adapted as per our needs: https://www.youtube.com/watch?v=Uc6qBg7mM2Y 

In [69]:
%pip install gym[clasic_control]
%pip install pygame


Note: you may need to restart the kernel to use updated packages.


#Setup Environment#

In [70]:
import gym
import tensorflow as tf
from tensorflow import keras
import numpy as np 

import time 
import math 
import random
from collections import deque

In [71]:
#Random seed initialisation#
RANDOM_SEED = 5
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [72]:
#Set up Environment
env = gym.make("CartPole-v1", render_mode="rgb_array")

print("Action Space: {}".format(env.action_space))
print("State space: {}".format(env.observation_space))

Action Space: Discrete(2)
State space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


#Parameters#

In [73]:
#Parameters#
learning_rate = 0.7
discount_factor = 0.618

epsilon = 1
max_epsilon = 1
min_epsilon = 0.01
decay = 0.01

train_episodes = 300
test_episodes = 100

#Setup Deep Q Network#

In [74]:
#Deep RL Agent#
def agent(state_shape, action_shape):
    lr = 0.001 #initial learning rate
    init = tf.keras.initializers.HeUniform(seed=RANDOM_SEED)
    model = keras.Sequential()
    model.add(keras.layers.Dense(24, input_shape=state_shape, activation='relu', kernel_initializer=init))
    model.add(keras.layers.Dense(12, activation='relu', kernel_initializer=init))
    model.add(keras.layers.Dense(action_shape, activation='linear', kernel_initializer=init))
    model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.Adam(learning_rate==lr), metrics=['accuracy'])
    return model

In [75]:
#Get Q values
def get_qs(model, state, step):
    return model.predict(state.reshape([1, state.shape[0]]))[0]

In [76]:
#Train our RL Agent#
#Q Update#
def train(env, replay_memory, model, target_model, done):

    MIN_REPLAY_SIZE = 1000
    if len(replay_memory) < MIN_REPLAY_SIZE:
        return

    batch_size = 64 * 2
    mini_batch = random.sample(replay_memory, batch_size)
    current_states = np.array([transition[0] for transition in mini_batch])
    current_qs_list = model.predict(current_states)
    new_current_states = np.array([transition[3] for transition in mini_batch])
    future_qs_list = target_model.predict(new_current_states)

    X = []
    Y = []
    for index, (observation, action, reward, new_observation, done) in enumerate(mini_batch):
        if not done:
            max_future_q = reward + discount_factor * np.max(future_qs_list[index])
        else:
            max_future_q = reward

        current_qs = current_qs_list[index]
        current_qs[action] = (1 - learning_rate) * current_qs[action] + learning_rate * max_future_q

        X.append(observation)
        Y.append(current_qs)
        
    model.fit(np.array(X), np.array(Y), batch_size=batch_size, verbose=0, shuffle=True)

#Initialize Target and Main Models#

In [77]:
model = agent(env.observation_space.shape, env.action_space.n)
target_model = agent(env.observation_space.shape, env.action_space.n)
target_model.set_weights(model.get_weights())

In [78]:
replay_memory = deque(maxlen=50_000)
target_update_counter = 0
steps_to_update_target_model = 0

# X = states, y = actions
X = []
y = []

In [80]:
for episode in range(train_episodes):
        total_training_rewards = 0
        observation = env.reset()
        done = False

        while not done:
            steps_to_update_target_model += 1
            if True:
                env.render()

            random_number = np.random.rand()
            # Explore using the Epsilon Greedy Exploration Strategy
            if random_number <= epsilon:
                # Explore
                action = env.action_space.sample()
            else:
                # Exploit best known action
                encoded = np.asarray(observation)
                encoded_reshaped = encoded.reshape([1, encoded.shape[0]])
                predicted = model.predict(encoded_reshaped).flatten()
                
                action = np.argmax(predicted)
           
            new_observation, reward, done, truncated, info = env.step(action)
            replay_memory.append([observation, action, reward, new_observation, done])

            # Update the Main Network using the Bellman Equation
            if steps_to_update_target_model % 4 == 0 or done:
                train(env, replay_memory, model, target_model, done)

            observation = new_observation
            total_training_rewards += reward

            if done:
                print('Episode: {} , Training Reward: {}, Final Reward: {}'.format(episode, total_training_rewards, reward))
                total_training_rewards += 1

                if steps_to_update_target_model >= 100:
                    #print('Copying main network weights to the target network weights')
                    target_model.set_weights(model.get_weights())
                    steps_to_update_target_model = 0
                break

        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay * episode)

Episode: 0 , Training Reward: 13.0, Final Reward: 1.0
Episode: 1 , Training Reward: 18.0, Final Reward: 1.0
Episode: 2 , Training Reward: 24.0, Final Reward: 1.0
Episode: 3 , Training Reward: 22.0, Final Reward: 1.0
Episode: 4 , Training Reward: 18.0, Final Reward: 1.0
1/1 [==============================] - 0s 53ms/step
Episode: 5 , Training Reward: 22.0, Final Reward: 1.0


C:\Users\19255551\AppData\Local\Temp\ipykernel_26432\479210719.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  encoded = np.asarray(observation)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
#env.close()